#Install Dependencies

In [1]:
!pip install tensorflow==2.14 tensorflow-metal==1.1 opencv-python-headless matplotlib

In [2]:
import sys
print(sys.executable)

/Users/gabrielschull/anaconda3/envs/facialrecognition/bin/python


In [3]:
#import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt


In [4]:
#import tensorflow dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [5]:
#Set GPU Memory Consumption Growth to avoid OOM errors -> wrong tensorflow version for this, not necessary though
gpus = print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Num GPUs Available:  1


In [6]:
#Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [7]:
#Make dirs
#os.makedirs(POS_PATH)
#os.makedirs(NEG_PATH)
#os.makedirs(ANC_PATH)

In [8]:
#Collecting labelled faces in the wild dataset at http://vis-www.cs.umass.edu/lfw/ (as gzipped tar file)

In [9]:
#uncompress Tar Labelled Faces in the Wild dataser
#!tar -xf lfw.tar

In [10]:
# Move lfw imgs to the following repo data/negative
#for directory in os.listdir('lfw'):
   # for file in os.listdir(os.path.join('lfw', directory)):
       # EX_PATH = os.path.join('lfw', directory, file)
       # NEW_PATH = os.path.join(NEG_PATH, file)
       # os.replace(EX_PATH, NEW_PATH)
    

In [11]:
import uuid

In [12]:
#Collect positive and anchor classes

# cap = cv2.VideoCapture(0) #Establish a connection to the webcam
# while cap.isOpened():
#     ret, frame = cap.read() 
    
#     #cut down to 250x250px frame
#     frame = frame[250:250+250,475:475+250,:]

#     key = cv2.waitKey(1) & 0XFF

#     #collect anchors
#     if key == ord('a'):
#         #create unique filepath
#         imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
#         #write out anchor image
#         cv2.imwrite(imgname, frame)
#         break

#     #collect positives
#     if key == ord('p'):
#         #create unique filepath
#         imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
#         #write out positive image
#         cv2.imwrite(imgname, frame)
#         break
        
        
#     #show image back to screen
#     cv2.imshow('Image Collection', frame)

#     #breaking if q hit on keyboard after 1 ms
#     if key == ord('q'):
#         break
# #release webcam & close the image show frame
# cap.release()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

In [13]:
#plt.imshow(frame)

In [14]:
#Load and preprocess Images
# get image directories
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(100)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(100)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(100)

2024-02-28 11:24:30.229030: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-28 11:24:30.229094: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-28 11:24:30.229105: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-28 11:24:30.229484: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-28 11:24:30.229539: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
dir_test = anchor.as_numpy_iterator()

In [16]:
print(dir_test.next())

b'data/anchor/cb0691d2-d2a2-11ee-944e-563466da705a.jpg'


In [17]:
#Preprocessing - scale and resize

def preprocess(file_path):

    # read in image from file path
    byte_img = tf.io.read_file(file_path)
    # load in the image
    img = tf.io.decode_jpeg(byte_img)
    # preprocessing steps - resize image to be 105x105x3
    img = tf.image.resize(img, (105,105))
    # scale image to be between 0 and 1
    img = img / 255.0

    
    return img

In [18]:
#dataset.map(preprocess)

In [19]:
# Create labelled dataset
# (anchor, positive) =>  1,1,1,1,1
# (anchor, negative) => 0,0,0,0,0

In [20]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [21]:
samples = data.as_numpy_iterator()

In [22]:
example = samples.next()

In [23]:
example

(b'data/anchor/ccda33ec-d2a2-11ee-944e-563466da705a.jpg',
 b'data/positive/0bf032e8-d2a3-11ee-ab5a-563466da705a.jpg',
 1.0)

In [24]:
# Build Train and Test Partition
def preprocess_twin(input_img, validation_img, label):
     return (preprocess(input_img), preprocess(validation_img), label)

In [25]:
preprocess_twin(*example)

(<tf.Tensor: shape=(105, 105, 3), dtype=float32, numpy=
 array([[[0.5155107 , 0.4645303 , 0.3900205 ],
         [0.5423703 , 0.4908297 , 0.4163199 ],
         [0.5439932 , 0.48516965, 0.40711176],
         ...,
         [0.2450937 , 0.17805351, 0.12922066],
         [0.3038749 , 0.2332867 , 0.18622786],
         [0.28197724, 0.211389  , 0.15648705]],
 
        [[0.5273977 , 0.47641727, 0.40190747],
         [0.5313926 , 0.48011208, 0.40426174],
         [0.5510872 , 0.49226362, 0.40991068],
         ...,
         [0.18953587, 0.12651066, 0.0763706 ],
         [0.20514213, 0.13561429, 0.08807529],
         [0.282433  , 0.21240498, 0.16506605]],
 
        [[0.5416767 , 0.49247035, 0.41263843],
         [0.552381  , 0.5044818 , 0.41848743],
         [0.5474345 , 0.488611  , 0.40625808],
         ...,
         [0.20026673, 0.13600767, 0.09598944],
         [0.19299057, 0.12831803, 0.08311994],
         [0.21570854, 0.15118939, 0.10553112]],
 
        ...,
 
        [[0.76807845, 0.72494113

In [26]:
# Build dataLoader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [27]:
# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [28]:
data

<_ShuffleDataset element_spec=(TensorSpec(shape=(105, 105, None), dtype=tf.float32, name=None), TensorSpec(shape=(105, 105, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [29]:
train_samples = train_data.as_numpy_iterator()

In [30]:
train_sample = train_samples.next()

In [31]:
len(train_sample[0])

16

In [32]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [33]:
inp = Input(shape=(105,105,3), name='input_image')

In [34]:
c1 = Conv2D(64, (10,10), activation='relu')(inp)

In [35]:
m1 = MaxPooling2D(64,(2,2), padding='same')(c1)

In [36]:
c2 = Conv2D(128, (7,7), activation='relu')(m1)
m2 = MaxPooling2D(64,(2,2), padding='same')(c2)

In [37]:
c3 = Conv2D(128, (4,4), activation='relu')(m2)
m3 = MaxPooling2D(64,(2,2), padding='same')(c3)

In [38]:
c4 = Conv2D(256, (4,4), activation='relu')(m3)
f1 = Flatten()(c4)
d1 = Dense(4096, activation='sigmoid')(f1)

In [39]:
mod = Model(inputs=[inp], outputs=[d1], name='embedding')

In [40]:
mod.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2  (None, 48, 48, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 21, 21, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 18, 18, 128)       26

In [41]:
# Build embedding layer
def make_embedding():
    inp = Input(shape=(105,105,3), name='input_image')

    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64,(2,2), padding='same')(c1)

    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64,(2,2), padding='same')(c2)

    # Third block
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64,(2,2), padding='same')(c3)

    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')


In [42]:
embedding = make_embedding()

In [43]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 21, 21, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 18, 18, 128)       26

In [44]:
# Build distance Layer
# Siamese L1 Distance class
class L1Dist(Layer):

    # Init inheritance
    def __init__(self, **kwargs):
        super().__init__()
        
    # Compare similarity
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [45]:
l1 = L1Dist()

In [46]:
#l1(anchor_embedding, validation_embedding)

In [47]:
input_image = Input(name='input_img', shape=(105,105,3))
validation_image = Input(name='validation_img', shape=(105,105,3))

In [48]:
inp_embedding = embedding(input_image)
val_embedding = embedding(validation_image)

In [49]:
siamese_layer = L1Dist()

In [50]:
distances = siamese_layer(inp_embedding, val_embedding)

In [51]:
classifier = Dense(1, activation='sigmoid')(distances)

In [52]:
classifier

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_2')>

In [53]:
siamese_network = Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [54]:
siamese_network.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 105, 105, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 105, 105, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 embedding (Functional)      (None, 4096)                 3896044   ['input_img[0][0]',           
                                                          8          'validation_img[0][0]']      
                                                                                     

In [55]:
def make_siamese_model():

    # Anchor image inputs in network
    input_image = Input(name='input_img', shape=(105,105,3))
    # Validation image in network
    validation_image = Input(name='validation_img', shape=(105,105,3))

    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    # Classificaiton layer
    classifier = Dense(1, activation='sigmoid')(distances)

    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [56]:
siamese_model = make_siamese_model()

In [57]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 105, 105, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 105, 105, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 embedding (Functional)      (None, 4096)                 3896044   ['input_img[0][0]',           
                                                          8          'validation_img[0][0]']      
                                                                                     

In [58]:
#Training

In [59]:
# Set up Loss and Optimizer
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [60]:
# Define optimizer
opt = tf.keras.optimizers.legacy.Adam(1e-4)

In [61]:
# Establish checkpoints
checkpoint_dir = './training.checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [62]:
# Build train step function
@tf.function
def train_step(batch):
    with tf.device('/cpu:0'):
        with tf.GradientTape() as tape:
            # Get anchor and positive/negative image
            X = batch[:2]
            # Get label
            y = batch[2]

            # Forward pass
            yhat = siamese_model(X, training=True)
            #Calculate loss
            loss = binary_cross_loss(y, yhat)
        tf.print('loss:', loss)

        # Calculate gradients
        grad = tape.gradient(loss, siamese_model.trainable_variables)
        # Calculate updated weights and apply to siamese model
        opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        # Return loss
        return loss
    

In [63]:
# Build training 
def train(data, EPOCHS):
    
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step
            train_step(batch)
            progbar.update(idx+1)
            
        # Save checkpoints
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
    
            
        

In [64]:
#Train the model
EPOCHS = 50

In [ ]:
train(train_data, EPOCHS)

In [65]:
# Evaluate Model
# Import metric calculations
# Precision -> proportion of positive identifications were correct; Recall -> proportion of actual positives correctly identified
from tensorflow.keras.metrics import Precision, Recall

In [67]:
# Get a batch of test data
test_input_, test_val, y_true = test_data.as_numpy_iterator().next()

In [78]:
test_var = test_data.as_numpy_iterator().next()